# GONZAG WestMed example

This notebook demonstrates how you can blablabla

## Loading & initilizations

In [1]:
import sys
from os import getenv

GONZAG_DIR   = getenv('HOME')+'/DEV/gonzag'
CLIMPORN_DIR = getenv('HOME')+'/DEV/climporn/python'

sys.path.append(GONZAG_DIR)

import gonzag as gz
from gonzag.config import ldebug

GONZAG_DATA_DIR = '/MEDIA/data/GONZAG/gonzag_input'

# Satellite input data:
file_sat = GONZAG_DATA_DIR+'/SENTINEL3A_20170130-20170303.nc'
name_ssh_sat = 'sla_unfiltered'

# Model input data:
file_mod = GONZAG_DATA_DIR+'/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc'
name_ssh_mod = 'sossheig'
file_lsm_mod = file_mod; name_lsm_mod = '_FillValue' ; # we use _FillValue attribute of "nams_ssh_mod" in "file_mod"
l_griddist = False ; # grid is not strongly distorded





### Time overlap between model and satellite data ?

In [3]:
    (it1,it2), (Nts,Ntm) = gz.GetEpochTimeOverlap( file_sat , file_mod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2', it1,'--',it2)
    print('   => UTC: "'+gz.EpochT2Str(it1)+'" to "'+gz.EpochT2Str(it2)+'"\n')


 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 1485909000.0 -- 1488324600.0
   => UTC: "Wed Feb  1 00:30:00 2017" to "Tue Feb 28 23:30:00 2017"



## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [4]:
    clsm = name_lsm_mod
    if name_lsm_mod=='_FillValue': clsm = name_lsm_mod+'@'+name_ssh_mod
    
    ModelGrid = gz.ModGrid( file_mod, it1, it2, file_lsm_mod, clsm, distorded_grid=l_griddist )



 *** what we use to define model land-sea mask:
    => "_FillValue@sossheig" in "/MEDIA/data/GONZAG/gonzag_input/sossheig_box_WestMed_eNATL60-BLBT02_20170201-20170228.nc"

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (796, 868)
     * horizontal resolution:  0.016753984549605475  degrees or  1.8615352233066644  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [-180:180] frame...
     * lon_min, lon_max =  -5.5 9.53
     * lat_min, lat_max =  34.91 45.1
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  672
       ==> time record indices: 0 to 671, included



## Create object `SatelliteTrack` containing all the satellite track (aka _target_) info

In [5]:
SatelliteTrack = gz.SatTrack( file_sat, it1, it2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in /MEDIA/data/GONZAG/gonzag_input/SENTINEL3A_20170130-20170303.nc ...

 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  2452
       ==> time record indices: 95570 to 1410931, included



## Build the bilinear mapping & Perform the space-time interpolation

In [6]:
Solution = gz.Model2SatTrack( ModelGrid, name_ssh_mod, SatelliteTrack, name_ssh_sat )

 *** "found" distance criterion when searching for nearest point on model grid is  1.3961514174799983  km

 *** Will use zoom boxes of width of 269 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3961514174799983 134 )
      +++ Treated point: 122/2452 
          ==> Sat. coordinates:     38.617 4.061
      +++ Treated point: 244/2452 
          ==> Sat. coordinates:     40.024 2.34
      +++ Treated point: 366/2452 
          ==> Sat. coordinates:     41.67 9.441
      +++ Treated point: 488/2452 
          ==> Sat. coordinates:     41.581 4.013
      +++ Treated point: 610/2452 
          ==> Sat. coordinates:     41.09 3.632
      +++ Treated point: 732/2452 
          ==> Sat. coordinates:     44.556 -2.66
      +++ Treated point: 854/2452 
          ==> Sat. coordinates:     42.902 4.487
      +++ Treated point: 976/2452 
          ==> Sat. coordinates:     37.422 3.375
      +++ Treated point:

/home/laurent/DEV/gonzag/gonzag/bilin_mapping.py:108: RuntimeWarning: invalid value encountered in double_scalars
  zdalp = nmp.linalg.det( zM ) / zdeta
/home/laurent/DEV/gonzag/gonzag/bilin_mapping.py:111: RuntimeWarning: invalid value encountered in double_scalars
  zdbet = nmp.linalg.det( zM ) / zdeta


     ***    Done! *** 


 *** Starting space-time interpolation of model data onto the 2452 selected track points...
      jt = 00000 => satelite time = Wed Feb  1 09:56:04 2017
      jt = 00122 => satelite time = Wed Feb  1 21:16:00 2017
      jt = 00244 => satelite time = Sat Feb  4 10:19:42 2017
      jt = 00366 => satelite time = Sun Feb  5 09:53:03 2017
      jt = 00488 => satelite time = Sun Feb  5 21:13:07 2017
      jt = 00610 => satelite time = Wed Feb  8 10:15:40 2017
      jt = 00732 => satelite time = Wed Feb  8 21:36:24 2017
      jt = 00854 => satelite time = Thu Feb  9 21:09:45 2017
      jt = 00976 => satelite time = Sun Feb 12 10:12:58 2017
      jt = 01098 => satelite time = Mon Feb 13 21:04:57 2017
      jt = 01220 => satelite time = Thu Feb 16 10:07:47 2017
      jt = 01342 => satelite time = Thu Feb 16 21:27:30 2017
      jt = 01464 => satelite time = Sun Feb 19 10:31:50 2017
      jt = 01586 => satelite time = Mon Feb 20 10:05:15 2017
      jt = 01708 => satelite 

### Object `Solution` contains everything you need to start the plots and the science... To be continued soon!